In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hazsy\\Desktop\\Haz(Padhai Likhai)\\Cancer_MLFlow\\CANCER_CLASSIFICATION\\research'

In [3]:
os.chdir("../")

In [4]:
%ldir

 Volume in drive C is OS
 Volume Serial Number is 1240-461F

 Directory of c:\Users\hazsy\Desktop\Haz(Padhai Likhai)\Cancer_MLFlow\CANCER_CLASSIFICATION

29-12-2023  03:02    <DIR>          .
29-12-2023  03:02    <DIR>          ..
02-01-2024  16:10    <DIR>          .git
28-12-2023  18:53    <DIR>          .github
02-01-2024  18:27    <DIR>          artifacts
28-12-2023  18:53    <DIR>          config
28-12-2023  21:40    <DIR>          logs
30-12-2023  19:48    <DIR>          research
28-12-2023  23:33    <DIR>          src
28-12-2023  18:53    <DIR>          templates
               0 File(s)              0 bytes
              10 Dir(s)  23,433,293,824 bytes free


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    update_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
%pwd

'c:\\Users\\hazsy\\Desktop\\Haz(Padhai Likhai)\\Cancer_MLFlow\\CANCER_CLASSIFICATION'

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config=PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            update_base_model_path=Path(config.update_base_model_path),
        params_image_size=self.params.IMAGE_SIZE,
        params_learning_rate=self.params.LEARNING_RATE,
        params_include_top=self.params.INCLUDE_TOP,
        params_weights=self.params.WEIGHTS,
        params_classes=self.params.CLASSES
        )
        
        return prepare_base_model_config

In [9]:
# pip install tensorflow

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [11]:
class PrepareBaseModel:
    def __init__(self,config: PrepareBaseModelConfig):
        self.config= config 

    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        
        self.save_model(path=self.config.base_model_path,model=self.model)
        
    @staticmethod
    def _prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable=False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable=False
                
        flatten_in=tf.keras.layers.Flatten()(model.output)
        prediction=tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)
        
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
            
        )
        
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        
        self.save_model(path=self.config.update_base_model_path , model=self.full_model)
        
        
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [13]:
%pwd

'c:\\Users\\hazsy\\Desktop\\Haz(Padhai Likhai)\\Cancer_MLFlow\\CANCER_CLASSIFICATION'

In [14]:
print(CONFIG_FILE_PATH)

config\config.yaml


In [15]:
try:
    config = ConfigurationManager()
    prepare_base_model_config=config.get_prepare_base_model_config()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-01-03 11:15:41,936]:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-01-03 11:15:41,976]:INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-03 11:15:41,992]:INFO:common:created directory at: artifacts]
[2024-01-03 11:15:41,995]:INFO:common:created directory at: artifacts/prepare_base_model]
[2024-01-03 11:15:43,016]:WARNING:saving_utils:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


c:\Users\hazsy\anaconda3\envs\cancer\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 61, 61, 128)       0     